In [39]:
from unittest import result
import requests
import pandas as pd
import numpy as np
class Wordle:
    all = []
    parameters_endpoint = 'https://7b8uflffq0.execute-api.us-east-1.amazonaws.com/game/get_params'
    results_endpoint = 'https://7b8uflffq0.execute-api.us-east-1.amazonaws.com/game/check_results'
    def __init__(self,words_list,user:tuple):

        #add a check for the user


        # Run validations to the received arguments
        # assert price >= 0, f"Price {price} is not greater than or equal to zero!"
        
        # Assign to self object
        self.list_letter_index_incorrect = [] #check if this is the best way to do it
        self.wrong_letters = [] #check if this is the best way to do it
        #add posibles words for each step
        self.__num_attempts  = 0
        self.__attempt_list = []
        self.__last_attempt = ''
        self.__list_results = []
        self.user = user
        self.__parameters = self._get_parameters(user).json() 
        self.__id = self.__parameters['id']
        self.__length_word = self.__parameters['length_word']
        self.__vowels = self.__parameters['vowels']
        self.__consonants = self.__parameters['consonants']
        self.__words_list = words_list
        self.__df_game = self._get_df_game()
    @property
    def num_attempts(self):
        return self.__num_attempts #add funtionality that can't find the word in 6 attempps

    def _increase_num_attempts(self):
        self.__num_attempts += 1 

    @property
    def attempt_list(self):
        return self.__attempt_list
    
    def _append_attempt(self):
        self.__attempt_list.append(self.last_attempt)
        print(f"{self.last_attempt} - attempt #{self.num_attempts}")

    @property
    def last_attempt(self):
        return self.__last_attempt

    def _set_last_attempt(self,word):
        self.__last_attempt = word
    
    @property
    def list_results(self):
        return self.__list_results
    
    def _append_result(self,result):
        self.__list_results.append(result)
        print(result.json())

    def _get_parameters(self,user):
        """The format of the response is:
        {"id": "string",
        "length_word": int,
        "vowels": int,
        "consonants": int
        }
        """
        return requests.get(self.parameters_endpoint, auth = user)

    @property
    def parameters(self):
        return self.__parameters

    @property
    def id(self):
        return self.__id

    @property
    def length_word(self):
        return self.__length_word
    
    @property
    def vowels(self):
        return self.__vowels
    
    @property
    def consonants(self):
        return self.__consonants

    @property
    def words_list(self):
        return self.__words_list

    
    @property
    def df_game(self):
        return self.__df_game
    
    def _create_df_from_list(self):
        self.df_all_words = pd.DataFrame(self.words_list, columns =['Words'])

    def _add_number_letters_column(self):
        self.df_all_words['Number of letters'] = self.df_all_words['Words'].str.len()    
    
    def _add_vowels_column(self):
        self.df_all_words['Number of vowels'] = self.df_all_words['Words'].str.count(r'[aeiou]')

    def _add_consonant_column(self):
        self.df_all_words['Number of consonants'] = (self.df_all_words['Number of letters'] 
                                                     - self.df_all_words['Number of vowels'])
                                            
    def _restrict_df_by_parameter(self):
        conditions = (self.df_all_words['Number of letters'] == self.__length_word)  & (self.df_all_words['Number of vowels'] == self.__vowels) & (self.df_all_words['Number of consonants'] == self.consonants)
        return self.df_all_words.loc[conditions]

    def _get_df_game(self):
        self._create_df_from_list()
        self._add_number_letters_column()
        self._add_vowels_column()
        self._add_consonant_column()
        return self._restrict_df_by_parameter()

    def valid_words_list(self):
        return list(self.df_game['Words'].values)

    def word_from_list(self,list_words):
        new_attempt = max(list_words, key=lambda word: len(set(word)))
        return new_attempt


    def first_attempt(self):
        word = self.word_from_list(self.valid_words_list())
        return word


    def filter_position_letter(self, list_tupple):
        #add a comment

        df = self.df_game
        if list_tupple:
            bool_words = [df['Words'].str[index].eq(letter) for index, letter in list_tupple]
            list_words = list(df.loc[np.logical_and.reduce(bool_words), 'Words'].values)
            return list_words
        else: 
            return []
    
    def new_attempt(self):
        #index where the list of bools were true
        correct_index_letters_try = np.where(self.last_result_json['position_array'])[0]
        #getting index from letters that are in the word but are in wrong position
        index_letters_wrong_position = [index for index,letter in enumerate(self.last_attempt) if letter in self.last_result_json['right_letters_in_wrong_positions']] #cambiar 0 por current attempt
        #erase the index if are in the index of correct words
        incorrect_index_try = [index for index in index_letters_wrong_position if index not in correct_index_letters_try]
        #enumerate the atempt ex jorge = [(j,0),(o,1)]
        position_try = list(enumerate(self.last_attempt))
        #get the tuples that are right
        list_letter_index_correct = [position_try[j] for j in correct_index_letters_try]
        #get tupples that the letter exits on the word but are in wrong position
        self.list_letter_index_incorrect.extend([position_try[j] for j in incorrect_index_try])
        #list of letters that are in the correct position
        letter_correct_position = [self.last_attempt[j] for j in correct_index_letters_try]
        #list of letters that definitely aren't in the word
        wrong_letters_last_try = list(set(self.last_attempt) - set(self.last_result_json['right_letters_in_wrong_positions']) - set(letter_correct_position))
        right_letters = self.last_result_json['right_letters_in_wrong_positions']
        right_letters.extend(letter_correct_position)
        #right_letters = list(set(right_letters))#quitar esto en un futuro
        self.wrong_letters.extend(wrong_letters_last_try)
        if list_letter_index_correct:
            valid_words = self.filter_position_letter(list_letter_index_correct)
        else:
            valid_words = self.valid_words_list()
        invalid_words = self.filter_position_letter(self.list_letter_index_incorrect)
        valid_list = [word for word in valid_words if word not in invalid_words]
        a_list = [word for word in valid_list if all(invalid_letter not in word for invalid_letter in self.wrong_letters)]
        #valid letters mean must letters
        # final_list = [word for word in a_list if all(valid_letter in word for valid_letter in right_letters)]
        final_list = [word for word in a_list if all(right_letters.count(valid_letter) <= word.count(valid_letter) for valid_letter in right_letters)]

        new_try = self.word_from_list(final_list)
        return new_try

    def get_attempt(self):
        if self.num_attempts == 0:
            return self.first_attempt()
        else:
            return self.new_attempt()
    
    def update_data(self,word):
        self._increase_num_attempts()
        self._set_last_attempt(word)
        self._append_attempt()
        

    def send_attemp(self,word):
        self.update_data(word)
        result = requests.post(self.results_endpoint,json = {"result_word": word}, auth = self.user)
        self._append_result(result)
        self.last_result_json = result.json() 
        return result

    def play(self):
        #sending first attemp to the endpoint
        #print(self.first_attempt())
        self.send_attemp(self.first_attempt())
        # self.send_attemp1(self.new_attempt())
        print(self.new_attempt())
    

In [8]:
#from wordle import Wordle
import json

with open('/home/castilla/Desktop/sofka/reto_bi/OOP/test.txt', 'r') as f:
    list_words = json.loads(f.read())

user = ('jorge.castilla','5cbdaf7e3c844ec882f576ec2ec4c9a4')

game = Wordle(list_words,user)

In [9]:
game.parameters

{'id': '62b4da73a7523ed860dfabce',
 'length_word': 11,
 'vowels': 5,
 'consonants': 6}

In [10]:
first_attempt = game.get_attempt()
print(first_attempt)

bronquiales


In [11]:
game.send_attemp(first_attempt)


bronquiales - attempt #1
{'word_sent': 'bronquiales', 'score': 0.2727272727272727, 'try_datetime': '2022-06-23T21:28:15.365512', 'position_array': [False, False, False, False, False, False, True, False, False, False, False], 'right_letters_in_wrong_positions': ['o', 'n', 'e'], 'current_attemps': 1}


<Response [200]>

In [12]:
second_attemp = game.get_attempt()
print(second_attemp)

compitiendo


In [13]:
game.send_attemp(second_attemp)

compitiendo - attempt #2
{'word_sent': 'compitiendo', 'score': 1.0, 'try_datetime': '2022-06-23T21:30:24.889229', 'position_array': [True, True, True, True, True, True, True, True, True, True, True], 'right_letters_in_wrong_positions': [], 'current_attemps': 2}


<Response [200]>

In [14]:
game2 = Wordle(list_words,user)
game2.parameters

{'id': '62b4da73a7523ed860dfabce',
 'length_word': 11,
 'vowels': 5,
 'consonants': 6}

In [15]:
game2.parameters

{'id': '62b4dceeaf391d7254b85ad3',
 'length_word': 8,
 'vowels': 4,
 'consonants': 4}

In [16]:
first_attempt = game2.get_attempt()
print(first_attempt)

abiertos


In [17]:
game2.send_attemp(first_attempt)

abiertos - attempt #1
{'word_sent': 'abiertos', 'score': 0.375, 'try_datetime': '2022-06-23T21:39:00.007408', 'position_array': [True, False, True, False, False, False, False, False], 'right_letters_in_wrong_positions': ['t'], 'current_attemps': 1}


<Response [200]>

In [18]:
second_attemp = game2.get_attempt()
print(second_attemp)

adictiva


In [19]:
game2.send_attemp(second_attemp)

adictiva - attempt #2
{'word_sent': 'adictiva', 'score': 1.0, 'try_datetime': '2022-06-23T21:40:19.866921', 'position_array': [True, True, True, True, True, True, True, True], 'right_letters_in_wrong_positions': [], 'current_attemps': 2}


<Response [200]>

In [20]:
game3 = Wordle(list_words,user)
game3.parameters

{'id': '62b4de62af391d7254b85ad7',
 'length_word': 10,
 'vowels': 3,
 'consonants': 7}

In [21]:
first_attempt = game3.get_attempt()
print(first_attempt)

bialystock


In [24]:
game3.send_attemp(first_attempt)
second_attemp = game3.get_attempt()
print(second_attemp)

bialystock - attempt #2
{'word_sent': 'bialystock', 'score': 0.1, 'try_datetime': '2022-06-23T21:49:11.062416', 'position_array': [False, False, False, False, False, False, True, False, False, False], 'right_letters_in_wrong_positions': ['i', 'a', 's', 'o', 'c'], 'current_attemps': 3}
chortatzis


In [25]:
game3.send_attemp(second_attemp)

chortatzis - attempt #3
{'word_sent': 'chortatzis', 'score': 1.0, 'try_datetime': '2022-06-23T21:49:30.583059', 'position_array': [True, True, True, True, True, True, True, True, True, True], 'right_letters_in_wrong_positions': [], 'current_attemps': 4}


<Response [200]>

In [41]:
game4 = Wordle(list_words,user)
print(game4.parameters)
first_attempt = game4.get_attempt()
print(first_attempt)

{'id': '62b4e470a15bde70e02b0f6a', 'length_word': 11, 'vowels': 4, 'consonants': 7}
centigrados


In [42]:
game4.send_attemp(first_attempt)
second_attemp = game4.get_attempt()
print(second_attemp)

centigrados - attempt #1
{'word_sent': 'centigrados', 'score': 0.45454545454545453, 'try_datetime': '2022-06-23T22:09:51.688127', 'position_array': [False, True, False, False, False, False, False, True, False, False, False], 'right_letters_in_wrong_positions': ['n', 't', 'g', 'd', 'o', 's'], 'current_attemps': 1}
desgastando


In [43]:
game4.send_attemp(second_attemp)

desgastando - attempt #2
{'word_sent': 'desgastando', 'score': 1.0, 'try_datetime': '2022-06-23T22:10:26.086964', 'position_array': [True, True, True, True, True, True, True, True, True, True, True], 'right_letters_in_wrong_positions': [], 'current_attemps': 2}


<Response [200]>

In [44]:
game5 = Wordle(list_words,user)
print(game5.parameters)
first_attempt = game5.get_attempt()
print(first_attempt)

{'id': '62b4e470a15bde70e02b0f6a', 'length_word': 11, 'vowels': 4, 'consonants': 7}
abiertos


In [45]:
print(game5.parameters)

{'id': '62b4e50da15bde70e02b0f6e', 'length_word': 8, 'vowels': 4, 'consonants': 4}


In [46]:
game5.send_attemp(first_attempt)
second_attemp = game5.get_attempt()
print(second_attemp)

abiertos - attempt #1
{'word_sent': 'abiertos', 'score': 0.25, 'try_datetime': '2022-06-23T22:12:03.226984', 'position_array': [True, False, False, False, False, False, False, False], 'right_letters_in_wrong_positions': ['t', 'o'], 'current_attemps': 1}
actuando


In [47]:
game5.send_attemp(second_attemp)

actuando - attempt #2
{'word_sent': 'actuando', 'score': 0.375, 'try_datetime': '2022-06-23T22:12:12.449979', 'position_array': [True, False, False, False, False, False, True, True], 'right_letters_in_wrong_positions': ['t', 'a'], 'current_attemps': 2}


<Response [200]>

In [48]:
third_attemp = game5.get_attempt()
print(third_attemp)


adaptado


In [49]:
game5.send_attemp(third_attemp)

adaptado - attempt #3
{'word_sent': 'adaptado', 'score': 0.875, 'try_datetime': '2022-06-23T22:15:02.986128', 'position_array': [True, True, False, True, True, True, True, True], 'right_letters_in_wrong_positions': [], 'current_attemps': 3}


<Response [200]>

In [51]:
four_attemp = game5.get_attempt()
print(four_attemp)

ValueError: max() arg is an empty sequence